In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import os
import tqdm
import glob
from statistics import mode
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

In [2]:
def loadFileName(directory):
    dealerIDir = glob.glob(directory+'/*')
    allVidir = []
    for i in range(len(dealerIDir)):
        allVidir.append(glob.glob(dealerIDir[i]+'/*mp4'))
    return np.array(allVidir)
def eulidist(rectlist,center):
    dx1_2 = (rectlist[0]-center[0])**2
    dx2_2 = (rectlist[2]-center[0])**2
    dy1_2 = (rectlist[1]-center[1])**2
    dy2_2 = (rectlist[3]-center[1])**2
    return dx1_2+dx2_2+dy1_2+dy2_2
def selectFaces(faces,center):
    disList = []
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        loc = np.array((x,y,x+w,y+h))
        disList.append(eulidist(loc,center))
    disList = np.array(disList)
#     print(faces)
    return faces[np.argmin(disList)]

In [3]:
def extractFrame(mp4Dir):
    vid = cv2.VideoCapture(mp4Dir)
    allFrame = []
    while vid.isOpened():
        ret,frame = vid.read()
        if ret:
            allFrame.append(frame)
        else:
            break
    return np.array(allFrame)
def extractXYWH(allFrame):
    face_cascade = cv2.CascadeClassifier('C:/Users/44754/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    imgList = allFrame.copy()
    for i in tqdm.tqdm(range(len(allFrame))):
        img = imgList[i].copy()
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray)
        if not len(faces):
            face = np.array((0,0,0,0))
        else:
            center = np.array(imgList[i][:,:,::-1].shape)[:2][::-1]/2
            (x,y,w,h) = selectFaces(faces,center)
            face = np.array((x,y,w,h))
        if i == 0:
            coorList = face
        else:
            coorList = np.vstack((coorList,face))
    return coorList
def meanShiftCluster(coorList):
    ms = MeanShift(bandwidth=20)
    ms.fit(coorList[:,:2])
    labels = ms.labels_
    remainIdx = mode(labels)
    w = int(np.mean(coorList[labels==remainIdx,2]))
    h = int(np.mean(coorList[labels==remainIdx,3]))
    return labels,remainIdx,w,h
def plotCluster(coorList,labels):
    xmin,xmax = np.min(coorList[:,0]),np.max(coorList[:,0])
    ymin,ymax = np.min(coorList[:,1]),np.max(coorList[:,1])
    numCol = int(np.ceil((len(np.unique(labels))+1)/2))
    plt.figure(figsize=(20,10))
    plt.subplot(2,numCol,1)
    plt.scatter(coorList[:,0],coorList[:,1],c=labels)
    plt.xlim([xmin,xmax])
    plt.ylim([ymin,ymax])
    for i in range(len(np.unique(labels))):
        plt.subplot(2,numCol,i+2)
        plt.scatter(coorList[labels==i,0],coorList[labels==i,1])
        plt.xlim([xmin,xmax])
        plt.ylim([ymin,ymax])
        plt.title(np.sum(labels==i))
def modifyFrames(allFrame,labels,remainIdx,w,h):
    imgList = allFrame.copy()
    modList = []
    for i in tqdm.tqdm(range(len(allFrame))):
        img = imgList[i].copy()
        if labels[i] == remainIdx:
            x,y = coorList[i,0],coorList[i,1]
            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        modList.append(img)
    modList = np.array(modList)
    return modList

In [7]:
def extractFaces(allFrame,coorList,labels,remainIdx,w,h):
    imgList = allFrame.copy()
    facesList = []
    for i in range(len(allFrame)):
        if labels[i] == remainIdx:
            img = imgList[i].copy()
            x,y = coorList[i,0],coorList[i,1]
            tmp = img[y:y+h,x:x+w]
            facesList.append(tmp)
    facesList = np.array(facesList)
    return facesList
def dir2processedarr(mp4Dir):
    allFrame  = extractFrame(mp4Dir)
    print(allFrame.shape)
    coorArr = extractXYWH(allFrame)
    labels,remainIdx,w,h = meanShiftCluster(coorArr)
    facesArr = extractFaces(allFrame,coorArr,labels,remainIdx,w,h)
    return facesArr
def storeProcessedImgs(directory):
    currentdir = os.getcwd()
    # create train dir
    path = os.path.join(currentdir,'train')
    if not os.path.exists(path):
        os.makedirs(path)
    
    # list all IDs
    dealersID = os.listdir(directory)
    for i in range(len(dealersID)):
        print('{} dealers'.format(i))
        tmppath = os.path.join(path,dealersID[i])
        if not os.path.exists(tmppath):
            os.makedirs(tmppath)
        dealDir = os.path.join(directory,dealersID[i])
        vidsID = os.listdir(dealDir)
        j = 0
        # Each video for a specific dealer
        for vid in vidsID:
            mp4Dir = os.path.join(dealDir,vid)
            tmparr = dir2processedarr(mp4Dir)
            # saving
            for k in range(tmparr.shape[0]):
                savepath = os.path.join(tmppath,'{}.jpg'.format(j))
                cv2.imwrite(savepath,tmparr[k])
                j+=1

In [8]:
vidir = 'D:/DreamAI/videosubset'
storeProcessedImgs(vidir)

0 dealers
(2085, 540, 1024, 3)


100%|██████████████████████████████████████████████████████████████████████████████| 2085/2085 [03:12<00:00, 10.83it/s]


(1631, 540, 1024, 3)


100%|██████████████████████████████████████████████████████████████████████████████| 1631/1631 [02:49<00:00,  9.63it/s]


(1988, 540, 1024, 3)


...

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:52<00:00,  7.95it/s]


(745, 540, 1024, 3)


100%|████████████████████████████████████████████████████████████████████████████████| 745/745 [01:33<00:00,  7.95it/s]


(852, 540, 1024, 3)


100%|████████████████████████████████████████████████████████████████████████████████| 852/852 [01:46<00:00,  7.98it/s]


In [15]:
currentdir = os.getcwd()
# create train dir
path = os.path.join(currentdir,'train')
# list all IDs
dealersID = os.listdir(path)
for ids in dealersID:
    tmp = os.path.join(path,ids)
    print('Dealer {}: '.format(ids)+str(len(glob.glob(tmp+'/*.jpg'))))

Dealer 231: 15876
Dealer 634: 8803
Dealer 717: 16645
Dealer 818: 5979


### There are differences among the numbers of images for different dealers.